# ZymoBIOMICS Microbial Community Standards Sample Composition

The eight bacterial genomes and four plasmids of the ZymoBIOMICS Microbial Community Standards were used as reference. It contains tripled complete sequences for the following species:

- Bacillus subtilis
- Enterococcus faecalis
- Escherichia coli
  - Escherichia coli plasmid
- Lactobacillus fermentum
- Listeria monocytogenes
- Pseudomonas aeruginosa
- Salmonella enterica
- Staphylococcus aureus
  - Staphylococcus aureus plasmid 1
  - Staphylococcus aureus plasmid 2
  - Staphylococcus aureus plasmid 3

It also downloads the raw sequence data of the mock communities, with an even and logarithmic distribution of species:

- ERR2984773
- ERR2935805 

A set of simulated samples were generated from the genomes in the ZymoBIOMICS standard though the InSilicoSeq sequence simulator (version 1.5.2), including both even and logarithmic distribution, with and without Illumina error model. The number of read pairs generated matches the number of read pairs in the real data for each distribution. The following samples are available in Zenodo:

- ENN - Envenly distributed sample with no error model
- EHS - Envenly distributed sample with Illumina HiSeq error model
- LNN - Log distributed sample with no error model
- LHS - Log distributed sample with Illumina HiSeq error model


## Imports

In [1]:
import sys
from plotly.offline import plot
import glob
import fnmatch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import pandas as pd
from itertools import groupby
import csv
import numpy as np
import os

In [21]:
SPECIES = ['Bacillus subtilis', 'Enterococcus faecalis', 'Escherichia coli', 'Lactobacillus fermentum', 'Listeria monocytogenes', 
           'Pseudomonas aeruginosa', 'Salmonella enterica', 'Staphylococcus aureus']
COLOURS = ['#004B93', "#009392", "#39B185", "#9CCB86", "#E9E29C", "#EEB479", "#E88471","#CF597E",'lightgray', 'darkgray']

## Load Data

In [38]:
kraken_report_files = glob.glob('../Kraken/*.kraken_report')

kraken_data = {}
for report in kraken_report_files:
    sample = os.path.splitext(os.path.basename(report))[0]
    kraken_data[sample] = {}
    with open(report) as fh:
        for line in fh:
            line = line.split()
            if 'unclassified' in line[5]:
                kraken_data[sample]['unclassified'] = float(line[0])
            else:
                if 'Lactobacillus' in line[5]:
                    try:
                        if'fermentum' in line[6]:
                            #print(line)
                            kraken_data[sample][' '.join(line[5:7])] = round(float(line[0]),2)
                    except:
                        print(line)
                if line[3] == 'S':
                    if ' '.join(line[5:]) in SPECIES:
                        kraken_data[sample][' '.join(line[5:])] = round(float(line[0]),2)


    other = round(100 - sum(kraken_data[sample].values()),2)
    kraken_data[sample]['other']  = other

print(kraken_data)

['0.01', '3806', '97', 'G', '1578', 'Lactobacillus']
['0.01', '765', '22', 'G', '1578', 'Lactobacillus']
['0.01', '733', '10', 'G', '1578', 'Lactobacillus']
['0.00', '240', '13', 'G', '1578', 'Lactobacillus']
['0.01', '1158', '20', 'G', '1578', 'Lactobacillus']
['0.00', '2088', '38', 'G', '1578', 'Lactobacillus']
{'LHS': {'unclassified': 0.0, 'Escherichia coli': 31.52, 'Salmonella enterica': 3.43, 'Pseudomonas aeruginosa': 7.24, 'Staphylococcus aureus': 17.39, 'Bacillus subtilis': 0.99, 'Listeria monocytogenes': 1.89, 'Enterococcus faecalis': 8.52, 'Lactobacillus fermentum': 0.05, 'other': 28.97}, 'EHS': {'unclassified': 0.0, 'Staphylococcus aureus': 17.49, 'Bacillus subtilis': 1.44, 'Listeria monocytogenes': 8.06, 'Enterococcus faecalis': 8.51, 'Lactobacillus fermentum': 0.05, 'Escherichia coli': 14.0, 'Salmonella enterica': 7.31, 'Pseudomonas aeruginosa': 6.46, 'other': 36.68}, 'ENN': {'unclassified': 0.0, 'Staphylococcus aureus': 17.65, 'Bacillus subtilis': 1.42, 'Listeria monocytog

## Plot Data

In [39]:
fig_kraken = make_subplots(rows=3, cols=2, shared_xaxes=True, x_title="Species composition", 
                            shared_yaxes=True, y_title='PLS',
                            subplot_titles=('LNN', 'ENN', 'LHS', 'EHS', 'ERR2935805', 'ERR2984773'))

In [40]:
fig_kraken = go.Figure()
x = ['LNN', 'ENN', 'LHS', 'EHS', 'ERR2935805', 'ERR2984773']
ydict={}
for sample in x:
    for reference in kraken_data[sample]:
        if reference not in ydict:
            ydict[reference] = [kraken_data[sample][reference]]
        else:
            ydict[reference].append(kraken_data[sample][reference])
i=0
for reference in sorted(ydict): 
    fig_kraken.add_trace(go.Bar(x=x, y=ydict[reference], name=reference, marker_color=COLOURS[i], text=ydict[reference], textposition='auto'))
    i += 1
fig_kraken.update_layout(plot_bgcolor='rgb(255,255,255)', title_text="Taxonomic composition of the ZymoBIOMICS microbial community standards samples")
fig_kraken.update_layout(barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':['ENN', 'EHS', 'ERR2984773','LNN', 'LHS','ERR2935805']})
fig_kraken.show()
plot(fig_kraken, filename='Plots/Taxonomic composition.html', auto_open=False)

'Plots/Taxonomic composition.html'

In [28]:
SPECIES - ydict.keys()

set()

In [45]:
plot(fig_kraken, filename='Plots/Taxonomic composition.png', auto_open=False)

'Plots/Taxonomic composition.png.html'